In [2]:
!pip install laserembeddings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 kB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883961 sha256=6d2bc053cd16c5c9d1405e0a1b1419130fb55b7893081393e1285cd4734ddcd1
  Stored in directory: /root/.cache/pip/wheels/38/dd/61/9feb9767a39c2d6683b4cf2

In [3]:
!pip install --upgrade numpy==1.24.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 45.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.21.1 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
blosc2 3.2.0 requires numpy>=1.26, but you have numpy 1.24.3 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.3 which is incompatible.
accelerate 1.5.2 requires torch>=2.0.0, but you have torch 1.13.1 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompat

In [1]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [2]:
from laserembeddings import Laser
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [3]:
from warnings import filterwarnings
filterwarnings('ignore')

In [4]:
import re

import re

def read_data(file_path):

    sentences = []
    labels = []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            parts = line.rsplit("\t", 2)

            if len(parts) < 2:
                continue

            sentence = parts[0]
            label_str = parts[1]

            try:
                label = float(label_str)
            except ValueError:
                continue

            sentences.append(sentence)
            labels.append(label)

    return sentences, labels


def train_laser_classifier(train_file, test_file, lang='en'):

    train_sentences, train_labels = read_data(train_file)
    test_sentences, test_labels = read_data(test_file)

    train_labels = np.array(train_labels).astype(int)
    test_labels = np.array(test_labels).astype(int)

    laser = Laser()

    train_embeddings = laser.embed_sentences(train_sentences, lang=lang)
    test_embeddings = laser.embed_sentences(test_sentences, lang=lang)

    clf = LogisticRegression(max_iter=1000)
    clf.fit(train_embeddings, train_labels)

    predictions = clf.predict(test_embeddings)
    print("Classification Report on Test Data:")
    print(classification_report(test_labels, predictions))

In [ ]:
if __name__ == "__main__":
    train_laser_classifier("enma_natural_test.txt", "enma_natural_train.txt", lang='en')